<a href="https://colab.research.google.com/github/RaysirChang/-Coursera_Capstone/blob/master/Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To create the dataframe

In [1]:
import pandas as pd
import numpy as np
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') 
df=df[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [2]:
df.shape

(287, 3)

### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [4]:
df1=df.groupby(['Postcode'],sort=False).agg(','.join)
df1=df1.reset_index()
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,"North York,North York","Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [5]:
df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,"North York,North York","Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


#### remove the duplicate boroughts

In [6]:
df1['Borough']= df1['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,NorthYork,Parkwoods
1,M4A,NorthYork,Victoria Village
2,M5A,DowntownToronto,Harbourfront
3,M6A,NorthYork,"Lawrence Heights,Lawrence Manor"
4,M7A,DowntownToronto,Queen's Park


### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [7]:
df1.shape

(103, 3)

### Use the Geocoder package or the csv file to create the dataframe

In [8]:
url='http://cocl.us/Geospatial_data'
df_geo=pd.read_csv(url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Make the column name the same

In [0]:
df_geo.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df1.rename(columns={'Postcode':'PostalCode'},inplace=True)

### Merge the tables

In [10]:
df_res=df1.merge(df_geo,on ='PostalCode',how = 'inner')
df_res.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,NorthYork,Parkwoods,43.753259,-79.329656
1,M4A,NorthYork,Victoria Village,43.725882,-79.315572
2,M5A,DowntownToronto,Harbourfront,43.654260,-79.360636
3,M6A,NorthYork,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,DowntownToronto,Queen's Park,43.662301,-79.389494


In [11]:
df_res.shape

(103, 5)

### Cluster Neighborhoods

In [0]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

Run *k*-means to cluster the neighborhood into 5 clusters.

In [13]:
# set number of clusters
kclusters = 5

#df_res_clustering = df_res.drop('Neighbourhood', 1)
df_res_clustering = df_res[['Latitude','Longitude']]
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_res_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 2, 0, 2, 1, 3, 4, 4, 2], dtype=int32)

In [14]:
# add clustering labels
df_res.insert(0, 'Cluster Labels', kmeans.labels_)

#manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_res.head() # check the last columns!

,Cluster Labels,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,4,M3A,NorthYork,Parkwoods,43.753259,-79.329656
1,4,M4A,NorthYork,Victoria Village,43.725882,-79.315572
2,2,M5A,DowntownToronto,Harbourfront,43.654260,-79.360636
3,0,M6A,NorthYork,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,2,M7A,DowntownToronto,Queen's Park,43.662301,-79.389494


### Visualize the resulting clusters

In [15]:
#Taking postal code M5G location as an initial latitude, longitude value
latitude=df_res[df_res['PostalCode']=='M5G']['Latitude'].values[0]
longitude=df_res[df_res['PostalCode']=='M5G']['Longitude'].values[0]

# create map
map_clusters = folium.Map(location=[latitude, latitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_res['Latitude'], df_res['Longitude'], df_res['Neighbourhood'], df_res['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters